# 0. Import Libraries
Before we start, we will import all the libraries needed for this Student Notebook.

In [1]:
# Data Cleaning & Transformation
import pandas as pd
pd.set_option('display.max.columns', 99)
import numpy as np

# Data Understanding
from ydata_profiling import ProfileReport

# Data Visualisation
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning & Deep Learning
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

# 1.📝Data Preparation
---

> **OVERALL GOAL:** 
> - Ingest the data and relabelling the outcome variable

In [ ]:
train_df = pd.read_csv("data/lc_trainingset.csv")
test_df = pd.read_csv("data/lc_testset.csv")

train_df.sample(5)

## 1b. Determine train & test set dimension

We understand the dimensionality of the data using <code>.shape</code>

In [ ]:
print("Train", train_df.shape)
print("Test", test_df.shape)

In [ ]:
# Identify the missing column from test set
set(train_df.columns.tolist()) - set(test_df.columns.tolist())

<div class="alert alert-block alert-warning">
Notice that the test dataset has 1 variable less than the training dataset and is identified to be the loan_status, also know as the target variable

## 1c. Check Out Loan Status

We use <code>value_counts()</code> to see the number of instances of each unique status in the loan_status data column

In [ ]:
train_df['loan_status'].value_counts()

We visualise the loan status for ease of interpretation

<div class="alert alert-block alert-warning">
While our descriptive analysis shows that the outcome variable only has 2 different outcomes, according to the business rules, it is possible that the column that we are trying to predict for has other different outcomes. As a good data scientist, we need to deal with this possibility so that if a data with a new outcome occurs the model will be able to deal with it accordingly. So how do we go about this?</div>

## 1d. Relabelling the Loan Status

From the above visualisation, we observed that around 10 type of loan status exist in this data set. We are only interested in 2 status i.e. <b>Defaulted</b> and <b>Not Defaulted</b>. Hence, we will need to add a new variable which will be binary (0s and 1s).

- 0 means Not Defaulted
- 1 means Defaulted

All those loans, whose status is “Fully Paid”, “Current” will be categorized as Not Defaulted and anything else will be categorized as Defaulted. To achieve this we will introduce new variable defaulted.

In [ ]:
# First we define the function
def change_loan_status(loan_status):
    if loan_status in ['Fully Paid', 'Current']:
        return 0
    else:
        return 1

# Next we apply the function
train_df['loan_status'] = train_df['loan_status'].apply(change_loan_status)
train_df.head()

Once again, we apply the <code>value_counts()</code> to see the number of instances of each unique status in the loan_status data column. Now we can see that the loan status only has 2 category.

In [ ]:
train_df['loan_status'].value_counts()

<div class="alert alert-block alert-warning">
So in the real world, when your column to be predicted has more than 2 outcomes, and you wish you classify them into 2 different outcomes, the above is one of the approach you can take - using logic/domain-knowledge to categorise the classes together.</div>

In [ ]:
# Perform data cleaning. You may create as many cells as you need

In [ ]:
print(train_df.apply(lambda x: x.unique()))

In [ ]:
train_df2 = train_df.copy()
train_df2['issue_d'] = pd.to_datetime(train_df2['issue_d'], format = "%b-%Y").dt.date
train_df2['earliest_cr_line'] = pd.to_datetime(train_df2['earliest_cr_line'], format = "%b-%Y").dt.date
train_df2[['issue_d', 'earliest_cr_line']]
                

In [ ]:
train_df3 = train_df2.select_dtypes(include = ['object']).copy()
train_df3

# 2.🔍Exploratory Data Analysis
---

> **OVERALL GOAL:** 
> - Get an understanding for which variables are important, view summary statistics, and visualize the data

https://www.analyticsvidhya.com/blog/2021/08/how-to-perform-exploratory-data-analysis-a-guide-for-beginners/

In [ ]:
profile = ProfileReport(train_df)
profile.to_notebook_iframe()

## 2a. Visualising Missing Values

In [ ]:
train_df.info(verbose = True)

In [ ]:
cols = train_df.columns.to_list()

print('Missing Values in these columns:')
for col in cols:
    if len(train_df[train_df[col].isnull() == True]) != 0:
        print(col, "-",  train_df2[col].isnull().sum())
    else:
        pass

In [ ]:
train_df.describe().T

<div class="alert alert-block alert-info">
Doesnt seem possible for annual_inc, dti, revol_bal & revol_util to be 0. Hence, feature engineering needs to be implemented later

## 2b. Determine the distribution of loan status for categorical features

In [ ]:
train_df.select_dtypes('object').columns

In [ ]:
for i in train_df.term.unique():
    print(i)
    print(f"{train_df[train_df.term == i].loan_status.value_counts(normalize=True)}")
    print('==========================================')

In [ ]:
for i in train_df.emp_length.unique():
    print(i)
    print(f"{train_df[train_df.emp_length == i].loan_status.value_counts(normalize=True)}")
    print('==========================================')

<div class="alert alert-block alert-info">
Distribution of loan status is similar across varying years of employment, hence to be removed 

In [ ]:
for i in train_df.sub_grade.unique():
    print(i)
    print(f"{train_df[train_df.sub_grade == i].loan_status.value_counts(normalize=True)}")
    print('==========================================')

In [ ]:
for i in train_df.home_ownership.unique():
    print(i)
    print(f"{train_df[train_df.home_ownership == i].loan_status.value_counts(normalize=True)}")
    print('==========================================')

<div class="alert alert-block alert-info">
For "NONE" and "ANY", they dont provide much context, hence to group them under "OTHER". Furthermore, there are only 2 record of "ANY" in the training set, which is not a good representation. Hence to avoid biasness, it is best to regroup under "OTHER"

In [ ]:
for i in train_df.verification_status.unique():
    print(i)
    print(f"{train_df[train_df.verification_status 	 == i].loan_status.value_counts(normalize=True)}")
    print('==========================================')

<div class="alert alert-block alert-info">
Verified and Source Verified have similar distribution of status loan and by nature both meant the same thing, hence to group them together in feature engineering

In [ ]:
for i in train_df.purpose.unique():
    print(i)
    print(f"{train_df[train_df.purpose== i].loan_status.value_counts(normalize=True)}")
    print('==========================================')

In [ ]:
for i in train_df.initial_list_status.unique():
    print(i)
    print(f"{train_df[train_df.initial_list_status== i].loan_status.value_counts(normalize=True)}")
    print('==========================================')

<div class="alert alert-block alert-info">
Distribution of loan status is similar across varying different initial listing status of loan, hence to be removed

In [ ]:
for i in train_df.application_type.unique():
    print(i)
    print(f"{train_df[train_df.application_type== i].loan_status.value_counts(normalize=True)}")
    print('==========================================')

## 2c. Determine any collinearity between numeric features

In [ ]:
train_df['loan_status'] = train_df['loan_status'].apply(str)

corr = train_df.corr().round(3)
fig = px.imshow(corr, color_continuous_scale = 'plasma', text_auto = True, aspect = 'auto')

fig.show()

<div class="alert alert-block alert-info">

From the heat map, we can deduce strong <b>positive</b> correlation (>=0.5) between different variables:
- loan_amt ↔ installment
- total_acc ↔ open_acc
- pub_rec ↔ pub_rec_bankruptcies
    
There is an absence of strong <b>negative</b> correlation between variables

## 2d. Understand the Distribution of Loan Status with Correlated Variables
### i) loan_amt ↔ installment

<div class="alert alert-block alert-info">

Loan amount is the total sum borrowed whereas installment is the partial payment made towards repaying the loan amount, hence the high correlation and the possibility of duplicated column names <br>
    
Hence, let's study the distribution of these 2 variables with the outcome

In [ ]:
fig = px.scatter(train_df, x='installment', y='loan_amnt', color='loan_status', template='simple_white')
fig.show()

<div class="alert alert-block alert-info">

It is not easy to tell from the scatter plot as the dataset is unbalanced, having more records which have either fully paid their loan

In [ ]:
train_df['loan_status'] = train_df['loan_status'].apply(str)


nbins_loan = round((np.max(train_df['loan_amnt']) - np.min(train_df['loan_amnt']))/1000)
nbins_instal = round((np.max(train_df['installment']) - np.min(train_df['installment']))/50)

trace1 = px.histogram(train_df, x = 'loan_amnt', color = 'loan_status',  height = 400, width = 600)
trace2 = px.histogram(train_df, x = 'installment', color = 'loan_status', height = 400, width = 600)

trace1.show()
trace2.show()

<div class="alert alert-block alert-info">
It is observed that there is a clearer normal distribution for instalment where the peak is when instalment is around 300 - 400 dollars

### ii) Total_acc & Open_acc

In [ ]:
train_df['loan_status'] = train_df['loan_status'].apply(str)

trace1 = px.histogram(train_df, x = 'total_acc', color = 'loan_status',  height = 300, width = 500)
trace2 = px.histogram(train_df, x = 'open_acc', color = 'loan_status', height = 300, width = 500)

trace1.show()
trace2.show()

<div class="alert alert-block alert-info">
Similar frequency distribution between the 2 variables, hence they can be considered as dependent variables.
    
However, both loan status peak around the same value for each variables, hence may not serve as good features.

### iii) Pub_rec_bankruptcies and Pub_rec

In [ ]:
train_df['loan_status'] = train_df['loan_status'].apply(str)

trace1 = px.histogram(train_df, x = 'pub_rec_bankruptcies', color = 'loan_status',  height = 500, width = 700)
trace2 = px.histogram(train_df, x = 'pub_rec', color = 'loan_status', height = 500, width = 700)

trace1.show()
trace2.show()

# 3.🔄 Feature Engineering
---

> **OVERALL GOAL:** 
> - Select relevant features and enhance them to improve the overall performance of the machine learning model

First we convert the text data into useful categorical numerical variable

## 3a. Missing Values

<div class="alert alert-block alert-info">
Based on earlier EDA, we identify 6 columns consists of missing values. (See Section 2a)
   
<b><br><u> Dropping of columns with missing values </b></u>
<br>Out of the 6, `emp_title` and `title` have high cardinality which will not be considered as useful features to be learned by model. As mentioned previously, normal distribution for `loan_status` across varying `emp_length` are similar. Hence these columns to be removed. (See Section 2b)
    
<b><br><u> Imputation of columns with missing values </b></u>
<br>`revol_util`, `mort_ac`c and `pub_rec_bankruptcies` are numeric variables, but are right-skewed. Hence to impute using median


In [ ]:
train_df2 = train_df.copy()
train_df2['revol_util'] = train_df2['revol_util'].fillna(train_df2['revol_util'].median())
train_df2['mort_acc'] = train_df2['mort_acc'].fillna(train_df2['mort_acc'].median())
train_df2['pub_rec_bankruptcies'] = train_df2['pub_rec_bankruptcies'].fillna(train_df2['pub_rec_bankruptcies'].median())

In [ ]:
cols = train_df2.columns.to_list()

print('Missing Values in these columns:')
for col in cols:
    if len(train_df[train_df2[col].isnull() == True]) != 0:
        print(col, "-",  train_df2[col].isnull().sum())
    else:
        pass

## 3b. Treating of Categorical Features

In [ ]:
train_df2.select_dtypes('object').columns

### i) Label Encoding for Ordinal Features

In [ ]:
train_df2['term'] = train_df2['term'].str.strip().str[:2]
train_df2['term'].unique()

<div class="alert alert-block alert-info">
sub_grade is the subset of grade, hence sub_grade to be kept and discard grade which may cause multicollinearity

In [ ]:
train_df2 = train_df2.sort_values(['sub_grade'])
train_df2['sub_grade'].unique()

labelencoder = LabelEncoder()

train_df2['sub_grade'] = labelencoder.fit_transform(train_df2['sub_grade'])

### iii) Date Extraction

In [ ]:
train_df2['Year_Issued'] = pd.to_datetime(train_df2['issue_d'], format = '%b-%Y').dt.year
train_df2['Year_Issued'].value_counts()

In [ ]:
train_df2['Year_cr_line'] = pd.to_datetime(train_df2['earliest_cr_line'], format = '%b-%Y').dt.year
train_df2['Year_cr_line'].value_counts()

<div class="alert alert-block alert-info">
There seems to be missing data for the earlier parts of the years for both `issue_d` and `earliest_cr_line`, hence to drop these columns for incomplete covereage

### iv) Converting address to postal code

In [ ]:
train_df2['postalcode'] = train_df2['address'].str.split(" ").str[-1]
train_df2[['address', 'postalcode']]

In [ ]:
train_df2['postalcode'].value_counts()

### v) Dropping of Unnecessary Categorical Features

In [ ]:
pseudo_df = train_df2.drop(['id', 'grade', 'emp_title', 'emp_length', 'initial_list_status', 'address', 'title',
                            'Year_Issued', 'earliest_cr_line','issue_d'], axis =1)
pseudo_df.select_dtypes('object').columns

### vi) One Hot Encoding

In [ ]:
dummies = pseudo_df.select_dtypes('object').columns.tolist()
dummies.remove('loan_status') # have to do on separate row, cant put concurrently abv. remove mutates the list in-place
dummies

In [ ]:
train_df3 = train_df2.copy()
train_df3['verification_status'] = np.where(train_df3['verification_status'] == 'Source Verified', 'Verified', train_df3['verification_status'])
train_df3['home_ownership'].replace(['NONE', 'ANY'], 'OTHER', inplace = True)

train_df3['home_ownership'] = train_df3['home_ownership'].str.title()
train_df3['application_type'] = train_df3['application_type'].str.title()

for i in dummies:
    train_df3[i] = train_df3[i].str.title()
    print(train_df3[i].unique())

In [ ]:
train_df4 = pd.get_dummies(train_df3, columns = dummies, drop_first = True)
# train_df4.drop(dummies, axis  = 1, inplace = True)
train_df4

## 3c. Treating of Numeric Features

In [ ]:
train_df4.select_dtypes(['int64','float64']).columns

### annual_inc
As mentioned in Section 2a, not logical that borrower could borrow money with annual income = 0. Hence, to impute using median

Additionally, the annual income is extremely right-skewed with a wide range, hence to execute log transform

In [ ]:
train_df4[train_df4['annual_inc']==0]

In [ ]:
train_df4['annual_inc'].replace(0, train_df4['annual_inc'].median(), inplace = True)
if len(train_df4[train_df4['annual_inc']==0]) == 0:
    print("Imputation Successful")
else:
    raise Exception("Imputation Unsucessful")

In [ ]:
train_df4['annual_inc_log'] = (train_df4['annual_inc']).transform(np.log) # no need to +1 as 0 has been imputed
train_df4[['annual_inc_log', 'annual_inc']]

### total_acc & open_acc
`total_acc` is highly correlated with `open_acc` as shown in the heatmap under Section 2c. Hence, one of the columns has to be removed to minimise error arising from multicollinearity. As `total_acc` distribution is more than than `open_acc`, `open_acc` will be dropped. 

In [ ]:
train_df4.drop_duplicates(['total_acc', 'open_acc'])[['total_acc', 'open_acc']].sort_values(['total_acc', 'open_acc']).head(10)


### pub_rec_bankruptcies & pub_rec
Similar to above,  `pub_rec_bankruptcies` and `pub_rec` are highly correlated and similar distribution as mentioned in section 2d, pub_rec will be selected as derogatory records can be an early tell-tale signs to determine whether borrower will default

In [ ]:
train_df4.drop_duplicates(['pub_rec_bankruptcies', 'pub_rec'])[['pub_rec', 'pub_rec_bankruptcies']].sort_values(['pub_rec', 'pub_rec_bankruptcies']).head(10)


### loan_amt & installment
To drop installment, for the same reason as above

In [ ]:
train_df4.drop_duplicates(['loan_amnt', 'installment'])[['loan_amnt', 'installment']].sort_values(['loan_amnt', 'installment']).head(10)


In [ ]:
train_df5 = train_df4.drop(['open_acc', 'annual_inc', 'pub_rec_bankruptcies', 'installment', 'id', 'grade', 'emp_title', 
                            'emp_length', 'initial_list_status', 'address', 'title',
                           'Year_Issued', 'earliest_cr_line','issue_d'], axis = 1)
train_df5.columns

### dti
Similar to annual_inc, does not make sense to have 0 for dti. 

As the dti data is skewed with range from 0.01 (ignoring 0) to 9999 (which may be an error), these outliers need to be removed. To only include data in the 1st to 99th percentile based on the report generated in Section 2 where the common values make up 99.3%

In [ ]:
(train_df5['dti']==0).sum()

In [ ]:
(train_df5['dti']==9999).sum()

In [ ]:
lower_lim =train_df5['dti'].quantile(0.01)
upper_lim =train_df5['dti'].quantile(0.99)


train_df6 = train_df5[(train_df5['dti'] < upper_lim) & (train_df5['dti'] > lower_lim)]
print("max:", np.max(train_df6['dti']), "\nmin:", np.min(train_df6['dti']))
print('Number of Records Removed:', (len(train_df5)-len(train_df6)))

### revol_bal
Similar to annual_inc, does not make sense to have 0 for `revol_bal`. 

As the `revol_bal` data is skewed with range from 1 (ignoring 0) to 1298783, these outliers need to be removed. To only include data in the 1st to 99th percentile based on the report generated in Section 2 where the common values make up 99.5%

In [ ]:
lower_lim =train_df6['revol_bal'].quantile(0.01)
upper_lim =train_df6['revol_bal'].quantile(0.99)


train_df7 = train_df6[(train_df6['revol_bal'] < upper_lim) & (train_df6['revol_bal'] > lower_lim)]
print("max:", np.max(train_df7['revol_bal']), "\nmin:", np.min(train_df7['revol_bal']))
print('Number of Records Removed:', (len(train_df6)-len(train_df7)))

### revol_util
Similar to annual_inc, does not make sense to have 0 for `revol_util`. 

As the `revol_bal` data is skewed with range from 0.01 (ignoring 0) to 892.3, these outliers need to be removed. To only include data in the 1st to 99th percentile based on the report generated in Section 2 where the common values make up 97.8%

In [ ]:
lower_lim =train_df7['revol_util'].quantile(0.01)
upper_lim =train_df7['revol_util'].quantile(0.99)


train_df8 = train_df7[(train_df7['revol_util'] < upper_lim) & (train_df7['revol_util'] > lower_lim)]
print("max:", np.max(train_df8['revol_util']), "\nmin:", np.min(train_df8['revol_util']))
print('Number of Records Removed:', (len(train_df7)-len(train_df8)))

In [ ]:
train_df8.head(1)

In [ ]:
test_df.head(1)

In [ ]:
list(set(train_df8.columns.tolist()) - set(test_df.columns.tolist()))

# 4.🤖 Models Building and Comparing Model Perforamance
---

> **OVERALL GOAL:** 
> - Building a machine learning model that is capable of generating predictions

## 4a. Logistic Regression (baseline model without any feature engineering)

Here we start off by creating a baseline model whhich you can use to compare against

<b>Selecting the feature columns</b><br>
We select the features we want to use in predicting our outcome

In [ ]:
feat_selection = train_df5.columns.tolist()
feat_selection.remove('loan_status')

X = train_df5[feat_selection] # Select the features you want to use to predict the loan_status
y = train_df5['loan_status'].astype(int)

# X_kaggle = test_df[feature_cols] # answers to this are hidden. you can't use these 759338 rows for model evaluation

<b> Import the models of your choice</b><br>
We select the features we want to use in predicting our outcome

In [ ]:
logreg_model = LogisticRegression(random_state=0)

<b> Train-Test Split</b></br>
We split the data to facilitate the evaluation of the model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

<div class="alert alert-block alert-warning">
You may opt for using k-fold cross validation as well.</div>

<b> Evaluate your model (using only lc_trainingset.csv)</b><br>
We generate a randomforest model by fitting the training data using <code>.fit()</code>, and thereafter generate predictions using <code>.predict()</code>

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
logreg_model = logreg_model.fit(X_train, y_train)
y_pred_proba = logreg_model.predict_proba(X_test)[:,1]

<b>We evaluate the model's AUC using <code>metrics.roc_auc_score()</code></b>

In [ ]:
print('AUC:', metrics.roc_auc_score(y_test, y_pred_proba)) 

## 4b. Random Forest (using bagging)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# bestEst = 0
# bestDepth = 0
# bestFeat = 0
# bestAUC = 0
# bestAcc = 0
# print("n_estimator | max_depth | max_features | AUC | Accuracy")

# for est in [100, 200]:
#     for depth in [2,4,8, None]:
#         for maxfeat in ['sqrt', 'log2', None]:
#             rf = RandomForestClassifier(n_estimators = est, max_depth = depth, max_features = maxfeat, random_state = 5)
#             rf.fit(X_train, y_train)
            
#             y_pred_proba= rf.predict_proba(X_test)[:,1]
#             y_predict = rf.predict(X_test)
            
#             auc = round(metrics.roc_auc_score(y_test, y_pred_proba), 4)
#             accuracy = round(metrics.accuracy_score(y_test, y_predict), 4)
            
#             print(est,  "|", depth, "|", maxfeat, "|", auc, "|", accuracy)
            
#             if accuracy > bestAcc:
#                 bestAUC = auc
#                 bestEst = est
#                 bestDepth = depth
#                 bestFeat = maxfeat
#                 bestAcc = accuracy
#             else:
#                 pass
            
# print(bestEst, "|", bestDepth, "|", bestFeat, "|", bestAUC, "|", bestAcc)

## 4c. XGBoost (using boosting)

In [ ]:
import xgboost as xgb

bestAUC = 0
bestdepth = 0
bestlr = 0
bestacc = 0
print("Learning Rate | Max Depth | AUC | Accuracy")
for lr in range(1,6):
    for depth in [2,4,8]:
        xgboost = xgb.XGBClassifier(learning_rate=lr/10, max_depth = depth, booster = 'gbtree',
                                    random_state=5)
        xgboost.fit(X_train, y_train)
        y_pred_proba= xgboost.predict_proba(X_test)[:,1]
        y_predict = xgboost.predict(X_test)
        accuracy = metrics.accuracy_score(y_test, y_predict)
        auc_score = metrics.roc_auc_score(y_test, y_pred_proba)
        print(lr/10, depth, round(auc_score,4), round(accuracy,4))
        if accuracy > bestacc:
            bestAUC = auc_score
            bestlr = lr/10
            bestdepth = depth
            bestacc = accuracy
        else:
            pass
print(bestlr, "|", bestdepth, "|", bestAUC, "|", bestacc)

In [ ]:
X

In [ ]:

feat_importances = pd.Series(xgboost.feature_importances_, index=X.columns).sort_values(ascending  = False)
fig = px.bar(feat_importances)
fig.show()

## 4d. Neural Network (using deep learning algorithm not taught in syllabus)

In [ ]:
# from sklearn.neural_network import MLPClassifier

# alpha = 0
# learing_rate = 0
# max_iteration = 0
# tol = 0
# n_iter = 0
# bestAUC = 0
# acc = 0

# print('Alpha | Learning Rate | Max Iteration | Tolerance | AUC | Accuracy')
# for a in [0.0001, 0.001]:
#     for learning_rate in range(1,5):
#         for max_iteration in [500, 1000]:
#             for tol in [0.001, 0.005]:
#                     perceptron = MLPClassifier(solver = 'adam', alpha = a,
#                                                learning_rate_init = (learning_rate/10), max_iter = max_iteration,
#                                                tol = tol,  random_state = 5)
#                     perceptron.fit(X_train, y_train.ravel())
#                     y_pred_proba= perceptron.predict_proba(X_test)[:,1]
#                     y_predict = perceptron.predict(X_test)
#                     auc_score = metrics.roc_auc_score(y_test, y_pred_proba)
#                     accuracy = metrics.accuracy_score(y_test, y_predict)
#                     print(a, learning_rate/10, max_iteration, tol, round(auc_score,4), round(accuracy, 4))
#                     if auc_score > bestAUC:
#                         bestAUC = round(auc_score,4)
#                         alpha = a
#                         learing_rate = learning_rate/10
#                         max_iteration = max_iteration
#                         tol = tol
#                         acc = round(accuracy,4)
#                     else:
#                         pass
# print(alpha, "|", learning_rate, "|", max_iteration, "|", tol, "|", bestAUC, "|", acc)

# 5.📊 Generate and Export Predictions from your Final Model
---

> **OVERALL GOAL:** 
> - Export your predictions and submit it to kaggle

## 5a. Re-fit your final model on train.csv

Say for instance you have determined the (based on accuracy)
1. optimal basket of features, 
2. the most optimal model for this dataset, and 
3. the best parameters for the model you have chosen

You should then retrain the chosen model with the optimal parameters, on the chosen basket of features on all the 316824 rows of data you have (instead of just on X_train, y_train). This is because you want to fully utilise your 316824 rows of data to maximise what your model can learn!

### Random Forest with tuned parameters for prediction

<div class="alert alert-block alert-info">
Random Forest model is selected as it gives the highest accuracy among the 3 models tested under Section 4 with an AUC > 0.85.

In [ ]:
feat_selection = train_df5.columns.tolist()
feat_selection.remove('loan_status')
feat_selection

In [ ]:
feat_selection = train_df5.columns.tolist()
feat_selection.remove('loan_status')

X = train_df5[feat_selection] # Select the features you want to use to predict the loan_status
y = train_df5['loan_status'].astype(int)
X = scaler.fit_transform(X)

xgboost = xgb.XGBClassifier(learning_rate=0.2, max_depth = 32, random_state=5)

xgboost.fit(X,y)
y_predict =xgboost.predict(X)
y_pred_proba= xgboost.predict_proba(X)[:,1]
accuracy = metrics.accuracy_score(y, y_predict)
auc = metrics.roc_auc_score(y, y_pred_proba)
print(accuracy, auc)

## 5b. Generate predictions for the test_df

Here, we will generate the predictions for the test_df. Make sure to apply whatever feature engineering technique you performed on train_df to test_df as well.

In [ ]:
# Replicate the feature engineering you did to the train_df
test_df = pd.read_csv('data/lc_testset.csv')
test_df.sample(2)

In [ ]:
# Impute missing values
test_df['mort_acc'] = test_df['mort_acc'].fillna(test_df['mort_acc'].median())

# Label encoding for term and subgrades
test_df['term'] = test_df['term'].str.strip().str[:2]

test_df = test_df.sort_values(['grade'])
test_df['sub_grade'].unique()
test_df['sub_grade'] = LabelEncoder().fit_transform(test_df['sub_grade'])
test_df.sort_index(inplace = True)

# Date Extraction
test_df['Year_Issued'] = pd.to_datetime(test_df['issue_d'], format = '%b-%Y').dt.year
test_df['Year_cr_line'] = pd.to_datetime(test_df['earliest_cr_line'], format = '%b-%Y').dt.year

# Address to Postal Code
test_df['postalcode'] = test_df['address'].str.split(" ").str[-1]

# One Hot Encoding
dummies = ['term', 'home_ownership', 'verification_status', 'purpose', 'application_type', 'postalcode']

test_df['verification_status'] = np.where(test_df['verification_status'] == 'Source Verified', 'Verified', test_df['verification_status'])
test_df['home_ownership'].replace(['NONE', 'ANY'], 'OTHER', inplace = True)
test_df['home_ownership'] = test_df['home_ownership'].str.title()
test_df['application_type'] = test_df['application_type'].str.title()
test_df['purpose'] = test_df['purpose'].str.title()

test_df = pd.get_dummies(test_df, columns = dummies, drop_first = True)

# Log Transform
test_df['annual_inc'].replace(0, test_df['annual_inc'].median(), inplace = True)
test_df['annual_inc_log'] = (test_df['annual_inc']).transform(np.log)

# # Remove outlier
# lower_lim =test_df['dti'].quantile(0.01)
# upper_lim =test_df['dti'].quantile(0.99)
# test_df = test_df[(test_df['dti'] < upper_lim) & (test_df['dti'] > lower_lim)]

# lower_lim =test_df['revol_bal'].quantile(0.01)
# upper_lim =test_df['revol_bal'].quantile(0.99)
# test_df = test_df[(test_df['revol_bal'] < upper_lim) & (test_df['revol_bal'] > lower_lim)]

# lower_lim =test_df['revol_util'].quantile(0.01)
# upper_lim =test_df['revol_util'].quantile(0.99)
# test_df = test_df[(test_df['revol_util'] < upper_lim) & (test_df['revol_util'] > lower_lim)]

test_df = test_df.drop(['open_acc', 'pub_rec_bankruptcies', 'installment', 'id', 'grade', 'emp_title', 'annual_inc', 
                            'emp_length', 'initial_list_status', 'address', 'title',
                           'Year_Issued', 'earliest_cr_line','issue_d'], axis = 1)
test_df

Here we use <code>.predict_proba</code> instead of <code>.predict()</code> because this is the format which the Kaggle platform requires

In [ ]:
kaggle_x = test_df #Extract the same features from the test set

kaggle_x = scaler.fit_transform(kaggle_x)
probabilities = xgboost.predict_proba(kaggle_x)

kaggle_preds = probabilities[:,1]  # extract values from the rightmost column
len(kaggle_preds)

Make use of the <code>.to_csv()</code> function to output your predictions in the form of a csv, which will be the format you will be required to submit to Kaggle.

In [ ]:
output_dataframe = pd.DataFrame({
    'id': list(range(len(kaggle_preds))),
    'Predicted': kaggle_preds
})
output_dataframe.to_csv('data/my_predictions_xgboost_proba.csv', index=False)  

# Check for the .csv in the same folder as your Jupyter Notebook
# Try uploading this .csv to the Kaggle competition!